In [1]:
import os
for dirname, _, filenames in os.walk("../../weights"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Imports

In [2]:
import gc

import sys
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import numpy as np
import pandas as pd
import pytorch_lightning as pl
import sklearn.metrics as metrics
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import yaml 
import joblib
from sklearn import linear_model
from pytorch_lightning import Trainer, seed_everything
from scipy.optimize import minimize
from tqdm import tqdm
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    DataCollatorWithPadding,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
)

config

In [3]:
with open("./configs/deberta_v3_base_512_reinit.yaml", encoding="utf-8") as f:
    cfg1 = yaml.safe_load(f)
cfg1["general"]["wandb_desabled"] = True
cfg1["model_save"] = False
cfg1["enable_checkpointing"] = False
cfg1["pl_params"]["precision"] = 32
#cfg1["model_name"] = "../input/deberta-v3-base"
cfg1['save_weight_folder'] = "../../weights/deberta_v3_base_512_reinit"

with open("./configs/deberta_v3_base_4096_reinit.yaml", encoding="utf-8") as f:
    cfg2 = yaml.safe_load(f)
cfg2["general"]["wandb_desabled"] = True
cfg2["model_save"] = False
cfg2["enable_checkpointing"] = False
cfg2["pl_params"]["precision"] = 32
#cfg2["model_name"] = "../input/deberta-v3-base"
cfg2['save_weight_folder'] = "../../weights/deberta_v3_base_4096_reinit"

with open("./configs/deberta_v3_large_512_reinit.yaml", encoding="utf-8") as f:
    cfg3 = yaml.safe_load(f)
cfg3["general"]["wandb_desabled"] = True
cfg3["model_save"] = False
cfg3["enable_checkpointing"] = False
cfg3["pl_params"]["precision"] = 32
#cfg3["model_name"] = "../input/deberta-v3-large"
cfg3['save_weight_folder'] = "../../weights/deberta_v3_large_512_reinit"

with open("./configs/deberta_v3_large_4096_reinit.yaml", encoding="utf-8") as f:
    cfg4 = yaml.safe_load(f)
cfg4["general"]["wandb_desabled"] = True
cfg4["model_save"] = False
cfg4["enable_checkpointing"] = False
cfg4["pl_params"]["precision"] = 32
#cfg4["model_name"] = "../input/deberta-v3-large"
cfg4['save_weight_folder'] = "../../weights/deberta_v3_large_4096_reinit"

with open("./configs/deberta_large_512_reinit.yaml", encoding="utf-8") as f:
    cfg5 = yaml.safe_load(f)
cfg5["general"]["wandb_desabled"] = True
cfg5["model_save"] = False
cfg5["enable_checkpointing"] = False
cfg5["pl_params"]["precision"] = 32
#cfg5["model_name"] = "../input/deberta-large"
cfg5['save_weight_folder'] = "../../weights/deberta_large_512_reinit"

with open("./configs/muppet_large_512_reinit.yaml", encoding="utf-8") as f:
    cfg6 = yaml.safe_load(f)
cfg6["general"]["wandb_desabled"] = True
cfg6["model_save"] = False
cfg6["enable_checkpointing"] = False
cfg6["pl_params"]["precision"] = 32
#cfg6["model_name"] = "../input/muppet-roberta-large"
cfg6['save_weight_folder'] = "../../weights/muppet_large_512_reinit"

with open("./configs/deberta_xlarge_512_reinit.yaml", encoding="utf-8") as f:
    cfg7 = yaml.safe_load(f)
cfg7["general"]["wandb_desabled"] = True
cfg7["model_save"] = False
cfg7["enable_checkpointing"] = False
cfg7["pl_params"]["precision"] = 32
#cfg7["model_name"] = "../input/debertaxlarge"
cfg7['save_weight_folder'] = "../../weights/deberta_xlarge_512_reinit"

with open("./configs/mpnet_base_512_reinit.yaml", encoding="utf-8") as f:
    cfg8 = yaml.safe_load(f)
cfg8["general"]["wandb_desabled"] = True
cfg8["model_save"] = False
cfg8["enable_checkpointing"] = False
cfg8["pl_params"]["precision"] = 32
cfg8['save_weight_folder'] = "../../weights/mpnet_base_512_reinit"

with open("./configs/deberta_v3_small_512_reinit.yaml", encoding="utf-8") as f:
    cfg9 = yaml.safe_load(f)
cfg9["general"]["wandb_desabled"] = True
cfg9["model_save"] = False
cfg9["enable_checkpointing"] = False
cfg9["pl_params"]["precision"] = 32
cfg9['save_weight_folder'] = "../../weights/deberta_v3_small_512_reinit"

with open("./configs/deberta_v3_xsmall_512_reinit.yaml", encoding="utf-8") as f:
    cfg10 = yaml.safe_load(f)
cfg10["general"]["wandb_desabled"] = True
cfg10["model_save"] = False
cfg10["enable_checkpointing"] = False
cfg10["pl_params"]["precision"] = 32
cfg10['save_weight_folder'] = "../../weights/deberta_v3_xsmall_512_reinit"

In [4]:
with open("./configs/deberta_v3_base_512_reinit_pseudo3.yaml", encoding="utf-8") as f:
    cfg1p = yaml.safe_load(f)
cfg1p["general"]["wandb_desabled"] = True
cfg1p["model_save"] = False
cfg1p["enable_checkpointing"] = False
cfg1p["pl_params"]["precision"] = 32
#cfg1p["model_name"] = "../input/deberta-v3-base"
cfg1p['save_weight_folder'] = "../../weights/deberta_v3_base_512_reinit_pseudo"

with open("./configs/deberta_v3_base_4096_reinit_pseudo.yaml", encoding="utf-8") as f:
    cfg2p = yaml.safe_load(f)
cfg2p["general"]["wandb_desabled"] = True
cfg2p["model_save"] = False
cfg2p["enable_checkpointing"] = False
cfg2p["pl_params"]["precision"] = 32
#cfg2p["model_name"] = "../input/deberta-v3-base"
cfg2p['save_weight_folder'] = "../../weights/deberta_v3_base_4096_reinit_pseudo"

with open("./configs/deberta_v3_large_512_reinit_pseudo3.yaml", encoding="utf-8") as f:
    cfg3p = yaml.safe_load(f)
cfg3p["general"]["wandb_desabled"] = True
cfg3p["model_save"] = False
cfg3p["enable_checkpointing"] = False
cfg3p["pl_params"]["precision"] = 32
#cfg3p["model_name"] = "../input/deberta-v3-large"
cfg3p['save_weight_folder'] = "../../weights/deberta_v3_large_512_reinit_pseudo"

with open("./configs/deberta_v3_large_4096_reinit_pseudo.yaml", encoding="utf-8") as f:
    cfg4p = yaml.safe_load(f)
cfg4p["general"]["wandb_desabled"] = True
cfg4p["model_save"] = False
cfg4p["enable_checkpointing"] = False
cfg4p["pl_params"]["precision"] = 32
#cfg4p["model_name"] = "../input/deberta-v3-large"
cfg4p['save_weight_folder'] = "../../weights/deberta_v3_large_4096_reinit_pseudo"

with open("./configs/deberta_large_512_reinit_pseudo.yaml", encoding="utf-8") as f:
    cfg5p = yaml.safe_load(f)
cfg5p["general"]["wandb_desabled"] = True
cfg5p["model_save"] = False
cfg5p["enable_checkpointing"] = False
cfg5p["pl_params"]["precision"] = 32
#cfg5p["model_name"] = "../input/deberta-large"
cfg5p['save_weight_folder'] = "../../weights/deberta_large_512_reinit_pseudo"

with open("./configs/muppet_large_512_reinit_pseudo.yaml", encoding="utf-8") as f:
    cfg6p = yaml.safe_load(f)
cfg6p["general"]["wandb_desabled"] = True
cfg6p["model_save"] = False
cfg6p["enable_checkpointing"] = False
cfg6p["pl_params"]["precision"] = 32
#cfg6p["model_name"] = "../input/muppet-roberta-large"
cfg6p['save_weight_folder'] = "../../weights/muppet_large_512_reinit_pseudo"

with open("./configs/deberta_xlarge_512_reinit_pseudo.yaml", encoding="utf-8") as f:
    cfg7p = yaml.safe_load(f)
cfg7p["general"]["wandb_desabled"] = True
cfg7p["model_save"] = False
cfg7p["enable_checkpointing"] = False
cfg7p["pl_params"]["precision"] = 32
#cfg7p["model_name"] = "../input/debertaxlarge"
cfg7p['save_weight_folder'] = "../../weights/deberta_xlarge_512_reinit_pseudo"

with open("./configs/deberta_v3_base_512_reinit_pseudo3_cls.yaml", encoding="utf-8") as f:
    cfg8p = yaml.safe_load(f)
cfg8p["general"]["wandb_desabled"] = True
cfg8p["model_save"] = False
cfg8p["enable_checkpointing"] = False
cfg8p["pl_params"]["precision"] = 32
#cfg8p["model_name"] = "../input/deberta-v3-base"
cfg8p['save_weight_folder'] = "../../weights/deberta_v3_base_512_reinit_pseudo3_cls"

with open("./configs/deberta_v3_base_512_reinit_pseudo3_wap_attention.yaml", encoding="utf-8") as f:
    cfg9p = yaml.safe_load(f)
cfg9p["general"]["wandb_desabled"] = True
cfg9p["model_save"] = False
cfg9p["enable_checkpointing"] = False
cfg9p["pl_params"]["precision"] = 32
#cfg9p["model_name"] = "../input/deberta-v3-base"
cfg9p['save_weight_folder'] = "../weights/deberta_v3_base_512_reinit_pseudo3_wap_attention"

with open("./configs/deberta_v3_base_512_reinit_pseudo3_wap_attention_3epoch.yaml", encoding="utf-8") as f:
    cfg10p = yaml.safe_load(f)
cfg10p["general"]["wandb_desabled"] = True
cfg10p["model_save"] = False
cfg10p["enable_checkpointing"] = False
cfg10p["pl_params"]["precision"] = 32
#cfg10p["model_name"] = "../input/deberta-v3-base"
cfg10p['save_weight_folder'] = "../weights/deberta_v3_base_512_reinit_pseudo3_wap_attention_3epoch"

In [5]:
with open("./configs/svr/albert_xxlarge_v2_512_svr.yaml", encoding="utf-8") as f:
    cfg1s = yaml.safe_load(f)
cfg1s["general"]["wandb_desabled"] = True
cfg1s["model_save"] = False
cfg1s["enable_checkpointing"] = False
cfg1s["pl_params"]["precision"] = 16
#cfg1s["model_name"] = "../input/deberta-v3-base"
cfg1s['save_weight_folder'] = "../../weights/albert_xxlarge_v2_512_svr"

with open("./configs/svr/bigbird_roberta_large_4096_svr.yaml", encoding="utf-8") as f:
    cfg2s = yaml.safe_load(f)
cfg2s["general"]["wandb_desabled"] = True
cfg2s["model_save"] = False
cfg2s["enable_checkpointing"] = False
cfg2s["pl_params"]["precision"] = 16
#cfg2s["model_name"] = "../input/deberta-v3-base"
cfg2s['save_weight_folder'] = "../../weights/bigbird_roberta_large_4096_svr"

with open("./configs/svr/deberta_large_512_svr.yaml", encoding="utf-8") as f:
    cfg3s = yaml.safe_load(f)
cfg3s["general"]["wandb_desabled"] = True
cfg3s["model_save"] = False
cfg3s["enable_checkpointing"] = False
cfg3s["pl_params"]["precision"] = 16
#cfg3s["model_name"] = "../input/deberta-v3-base"
cfg3s['save_weight_folder'] = "../../weights/deberta_large_512_svr"

with open("./configs/svr/deberta_v2_xlarge_512_svr.yaml", encoding="utf-8") as f:
    cfg4s = yaml.safe_load(f)
cfg4s["general"]["wandb_desabled"] = True
cfg4s["model_save"] = False
cfg4s["enable_checkpointing"] = False
cfg4s["pl_params"]["precision"] = 16
#cfg4s["model_name"] = "../input/deberta-v3-base"
cfg4s['save_weight_folder'] = "../../weights/deberta_v2_xlarge_512_svr"

with open("./configs/svr/deberta_v2_xxlarge_512_svr.yaml", encoding="utf-8") as f:
    cfg5s = yaml.safe_load(f)
cfg5s["general"]["wandb_desabled"] = True
cfg5s["model_save"] = False
cfg5s["enable_checkpointing"] = False
cfg5s["pl_params"]["precision"] = 16
#cfg5s["model_name"] = "../input/deberta-v3-base"
cfg5s['save_weight_folder'] = "../../weights/deberta_v2_xxlarge_512_svr"

with open("./configs/svr/deberta_v3_base_4096_svr.yaml", encoding="utf-8") as f:
    cfg6s = yaml.safe_load(f)
cfg6s["general"]["wandb_desabled"] = True
cfg6s["model_save"] = False
cfg6s["enable_checkpointing"] = False
cfg6s["pl_params"]["precision"] = 16
#cfg6s["model_name"] = "../input/deberta-v3-base"
cfg6s['save_weight_folder'] = "../../weights/deberta_v3_base_4096_svr"

with open("./configs/svr/deberta_v3_base_512_svr.yaml", encoding="utf-8") as f:
    cfg7s = yaml.safe_load(f)
cfg7s["general"]["wandb_desabled"] = True
cfg7s["model_save"] = False
cfg7s["enable_checkpointing"] = False
cfg7s["pl_params"]["precision"] = 16
#cfg7s["model_name"] = "../input/deberta-v3-base"
cfg7s['save_weight_folder'] = "../../weights/deberta_v3_base_512_svr"

with open("./configs/svr/deberta_v3_large_4096_svr.yaml", encoding="utf-8") as f:
    cfg8s = yaml.safe_load(f)
cfg8s["general"]["wandb_desabled"] = True
cfg8s["model_save"] = False
cfg8s["enable_checkpointing"] = False
cfg8s["pl_params"]["precision"] = 16
#cfg8s["model_name"] = "../input/deberta-v3-base"
cfg8s['save_weight_folder'] = "../../weights/deberta_v3_large_4096_svr"

with open("./configs/svr/deberta_v3_large_512_svr.yaml", encoding="utf-8") as f:
    cfg9s = yaml.safe_load(f)
cfg9s["general"]["wandb_desabled"] = True
cfg9s["model_save"] = False
cfg9s["enable_checkpointing"] = False
cfg9s["pl_params"]["precision"] = 16
#cfg9s["model_name"] = "../input/deberta-v3-base"
cfg9s['save_weight_folder'] = "../../weights/deberta_v3_large_512_svr"

with open("./configs/svr/deberta_xlarge_512_svr.yaml", encoding="utf-8") as f:
    cfg10s = yaml.safe_load(f)
cfg10s["general"]["wandb_desabled"] = True
cfg10s["model_save"] = False
cfg10s["enable_checkpointing"] = False
cfg10s["pl_params"]["precision"] = 16
#cfg10s["model_name"] = "../input/deberta-v3-base"
cfg10s['save_weight_folder'] = "../../weights/deberta_xlarge_512_svr"

with open("./configs/svr/electra_large_512_svr.yaml", encoding="utf-8") as f:
    cfg11s = yaml.safe_load(f)
cfg11s["general"]["wandb_desabled"] = True
cfg11s["model_save"] = False
cfg11s["enable_checkpointing"] = False
cfg11s["pl_params"]["precision"] = 16
#cfg11s["model_name"] = "../input/deberta-v3-base"
cfg11s['save_weight_folder'] = "../../weights/electra_large_512_svr"

with open("./configs/svr/funnel_xlarge_512_svr.yaml", encoding="utf-8") as f:
    cfg12s = yaml.safe_load(f)
cfg12s["general"]["wandb_desabled"] = True
cfg12s["model_save"] = False
cfg12s["enable_checkpointing"] = False
cfg12s["pl_params"]["precision"] = 16
#cfg12s["model_name"] = "../input/deberta-v3-base"
cfg12s['save_weight_folder'] = "../../weights/funnel_xlarge_512_svr"

with open("./configs/svr/longformer_large_4096.yaml", encoding="utf-8") as f:
    cfg13s = yaml.safe_load(f)
cfg13s["general"]["wandb_desabled"] = True
cfg13s["model_save"] = False
cfg13s["enable_checkpointing"] = False
cfg13s["pl_params"]["precision"] = 16
#cfg13s["model_name"] = "../input/deberta-v3-base"
cfg13s['save_weight_folder'] = "../../weights/longformer_large_4096"

with open("./configs/svr/muppet_large_512_svr.yaml", encoding="utf-8") as f:
    cfg14s = yaml.safe_load(f)
cfg14s["general"]["wandb_desabled"] = True
cfg14s["model_save"] = False
cfg14s["enable_checkpointing"] = False
cfg14s["pl_params"]["precision"] = 16
#cfg14s["model_name"] = "../input/deberta-v3-base"
cfg14s['save_weight_folder'] = "../../weights/muppet_large_512_svr"

with open("./configs/svr/roberta_large_512_svr.yaml", encoding="utf-8") as f:
    cfg15s = yaml.safe_load(f)
cfg15s["general"]["wandb_desabled"] = True
cfg15s["model_save"] = False
cfg15s["enable_checkpointing"] = False
cfg15s["pl_params"]["precision"] = 16
#cfg15s["model_name"] = "../input/deberta-v3-base"
cfg15s['save_weight_folder'] = "../../weights/roberta_large_512_svr"

In [6]:
with open("./configs/svr/deberta_v3_base_4096_reinit_svr_pseudo.yaml", encoding="utf-8") as f:
    cfg1sp = yaml.safe_load(f)
cfg1sp["general"]["wandb_desabled"] = True
cfg1sp["model_save"] = False
cfg1sp["enable_checkpointing"] = False
cfg1sp["pl_params"]["precision"] = 16
#cfg1sp["model_name"] = "../input/deberta-v3-base"
cfg1sp['save_weight_folder'] = "../../weights/deberta_v3_base_4096_reinit_svr_pseudo"

with open("./configs/svr/deberta_v3_base_512_reinit_svr_pseudo.yaml", encoding="utf-8") as f:
    cfg2sp = yaml.safe_load(f)
cfg2sp["general"]["wandb_desabled"] = True
cfg2sp["model_save"] = False
cfg2sp["enable_checkpointing"] = False
cfg2sp["pl_params"]["precision"] = 16
#cfg2sp["model_name"] = "../input/deberta-v3-base"
cfg2sp['save_weight_folder'] = "../../weights/deberta_v3_base_512_reinit_svr_pseudo"

with open("./configs/svr/deberta_v3_large_4096_reinit_svr_pseudo.yaml", encoding="utf-8") as f:
    cfg3sp = yaml.safe_load(f)
cfg3sp["general"]["wandb_desabled"] = True
cfg3sp["model_save"] = False
cfg3sp["enable_checkpointing"] = False
cfg3sp["pl_params"]["precision"] = 16
#cfg3sp["model_name"] = "../input/deberta-v3-base"
cfg3sp['save_weight_folder'] = "../../weights/deberta_v3_large_4096_reinit_svr_pseudo"

with open("./configs/svr/deberta_v3_large_512_reinit_svr_pseudo.yaml", encoding="utf-8") as f:
    cfg4sp = yaml.safe_load(f)
cfg4sp["general"]["wandb_desabled"] = True
cfg4sp["model_save"] = False
cfg4sp["enable_checkpointing"] = False
cfg4sp["pl_params"]["precision"] = 16
#cfg4sp["model_name"] = "../input/deberta-v3-base"
cfg4sp['save_weight_folder'] = "../../weights/deberta_v3_large_512_reinit_svr_pseudo"

with open("./configs/svr/deberta_xlarge_512_reinit_svr_pseudo.yaml", encoding="utf-8") as f:
    cfg5sp = yaml.safe_load(f)
cfg5sp["general"]["wandb_desabled"] = True
cfg5sp["model_save"] = False
cfg5sp["enable_checkpointing"] = False
cfg5sp["pl_params"]["precision"] = 16
#cfg5sp["model_name"] = "../input/deberta-v3-base"
cfg5sp['save_weight_folder'] = "../../weights/deberta_xlarge_512_reinit_svr_pseudo"

# 1128追加
with open("./configs/svr/deberta_v3_base_512_reinit_svr_pseudo_wap.yaml", encoding="utf-8") as f:
    cfg6sp = yaml.safe_load(f)
cfg6sp["general"]["wandb_desabled"] = True
cfg6sp["model_save"] = False
cfg6sp["enable_checkpointing"] = False
cfg6sp["pl_params"]["precision"] = 16

with open("./configs/svr/deberta_v3_base_512_reinit_svr_pseudo_wap_all.yaml", encoding="utf-8") as f:
    cfg7sp = yaml.safe_load(f)
cfg7sp["general"]["wandb_desabled"] = True
cfg7sp["model_save"] = False
cfg7sp["enable_checkpointing"] = False
cfg7sp["pl_params"]["precision"] = 16

with open("./configs/svr/deberta_v3_base_512_reinit_svr_pseudo_mean_pooling_concatenate.yaml", encoding="utf-8") as f:
    cfg8sp = yaml.safe_load(f)
cfg8sp["general"]["wandb_desabled"] = True
cfg8sp["model_save"] = False
cfg8sp["enable_checkpointing"] = False
cfg8sp["pl_params"]["precision"] = 16

with open("./configs/svr/deberta_v3_base_4096_reinit_svr_pseudo_wap.yaml", encoding="utf-8") as f:
    cfg9sp = yaml.safe_load(f)
cfg9sp["general"]["wandb_desabled"] = True
cfg9sp["model_save"] = False
cfg9sp["enable_checkpointing"] = False
cfg9sp["pl_params"]["precision"] = 16

with open("./configs/svr/deberta_v3_base_4096_reinit_svr_pseudo_wap_all.yaml", encoding="utf-8") as f:
    cfg10sp = yaml.safe_load(f)
cfg10sp["general"]["wandb_desabled"] = True
cfg10sp["model_save"] = False
cfg10sp["enable_checkpointing"] = False
cfg10sp["pl_params"]["precision"] = 16

with open("./configs/svr/deberta_v3_base_4096_reinit_svr_pseudo_mean_pooling_concatenate.yaml", encoding="utf-8") as f:
    cfg11sp = yaml.safe_load(f)
cfg11sp["general"]["wandb_desabled"] = True
cfg11sp["model_save"] = False
cfg11sp["enable_checkpointing"] = False
cfg11sp["pl_params"]["precision"] = 16

In [9]:
with open("./configs/deberta_v3_base_512_reinit_mse.yaml", encoding="utf-8") as f:
    cfg1mse = yaml.safe_load(f)
cfg1mse["general"]["wandb_desabled"] = True
cfg1mse["model_save"] = False
cfg1mse["enable_checkpointing"] = False
cfg1mse["pl_params"]["precision"] = 32

In [7]:
cfg_list = [cfg1, cfg2, cfg3, cfg4, cfg5, cfg6, cfg7]

In [8]:
cfg_list = [cfg1p, cfg2p, cfg3p, cfg4p, cfg5p, cfg6p, cfg7p]

In [9]:
cfg_list = [cfg1s, cfg2s, cfg3s, cfg4s, cfg5s, cfg6s, cfg7s, cfg8s, cfg9s, cfg10s, cfg11s, cfg12s, cfg13s, cfg14s, cfg15s]

In [10]:
cfg_list = [cfg1sp, cfg2sp, cfg3sp, cfg4sp, cfg5sp]

In [7]:
cfg_list = [cfg1, cfg2, cfg3, cfg4, cfg5, cfg6, cfg7, cfg1p, cfg2p, cfg3p, cfg4p, cfg5p, cfg6p, cfg7p, cfg1s, cfg2s, cfg3s, cfg4s, cfg5s, cfg6s, cfg7s, cfg8s, cfg9s, cfg10s, cfg11s, cfg12s, cfg13s, cfg14s, cfg15s, cfg1sp, cfg2sp, cfg3sp, cfg4sp, cfg5sp]

In [7]:
# 1126 sub
cfg_list = [cfg1, cfg2, cfg5, cfg6, cfg7, cfg1p, cfg3p, cfg4p, cfg7p, cfg2sp, cfg3sp, cfg4sp, cfg5sp]

In [23]:
# 1128 深夜
cfg_list = [cfg1, cfg2, cfg5, cfg6, cfg7, cfg1p, cfg3p, cfg4p, cfg7p, cfg2sp, cfg3sp, cfg4sp, cfg5sp]

In [5]:
cfg_list = [cfg1, cfg2, cfg3, cfg4, cfg5, cfg6, cfg7, cfg1p, cfg2p, cfg3p, cfg4p, cfg5p, cfg6p, cfg7p]

In [19]:
cfg_list = [cfg1, cfg2, cfg5, cfg6, cfg7, cfg1p, cfg3p, cfg4p, cfg7p] # best subしたやつ

In [27]:
cfg_list = [cfg1, cfg2, cfg5, cfg6, cfg7, cfg1p, cfg3p, cfg4p, cfg7p, cfg1s, cfg2s, cfg3s]

In [10]:
cfg_list = [cfg2, cfg5, cfg6, cfg7, cfg1p, cfg4p, cfg7p, cfg2sp, cfg3sp, cfg5sp, cfg8s, cfg3s, cfg6s]

In [11]:
cfg_list += [cfg1mse]

Model

In [12]:
def mcrmse(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:, i]
        y_pred = y_preds[:, i]
        score = metrics.mean_squared_error(y_true, y_pred, squared=False)  # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = (
            attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        )
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings


class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights=None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = (
            layer_weights
            if layer_weights is not None
            else nn.Parameter(
                torch.tensor(
                    [1] * (num_hidden_layers + 1 - layer_start), dtype=torch.float
                )
            )
        )

    def forward(self, all_hidden_states):
        all_layer_embedding = all_hidden_states[self.layer_start :, :, :, :]
        weight_factor = (
            self.layer_weights.unsqueeze(-1)
            .unsqueeze(-1)
            .unsqueeze(-1)
            .expand(all_layer_embedding.size())
        )
        weighted_average = (weight_factor * all_layer_embedding).sum(
            dim=0
        ) / self.layer_weights.sum()
        return weighted_average


class AttentionHead(nn.Module):
    def __init__(self, hidden_size):
        super(AttentionHead, self).__init__()
        self.att = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1),
            nn.Softmax(dim=1),
        )

    def forward(self, last_hidden_state):
        att_weights = self.att(last_hidden_state)
        feature = torch.sum(att_weights * last_hidden_state, dim=1)
        return feature


class TransformersModel(pl.LightningModule):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.criterion = nn.__dict__[cfg["criterion"]]()

        # awp
        if cfg["awp"] is not None:
            self.automatic_optimization = False
            self.adv_param = cfg["awp"]["adv_param"]
            self.adv_lr = cfg["awp"]["adv_lr"]
            self.adv_eps = cfg["awp"]["adv_eps"]
            self.adv_step = cfg["awp"]["adv_step"]
            self.backup = {}
            self.backup_eps = {}
            self.awp_accumulate_grad_batches = cfg["awp"]["accumulate_grad_batches"]
            # self.awp_scaler = torch.cuda.amp.GradScaler(enabled=cfg["awp"]["amp"])
            if cfg["awp"]["gradient_clip_val"] is not None:
                self.awp_max_grad_norm = cfg["awp"]["gradient_clip_val"]
            else:
                self.awp_max_grad_norm = None
            self.awp_start_epoch = cfg["awp"]["start_epoch"]

        # model
        if cfg["mlm"]:
            print(f"../weights/mlm_model/{cfg['model_name']}")
            self.tr_config = AutoConfig.from_pretrained(
                f"../weights/mlm_model/{cfg['model_name']}", output_hidden_states=True
            )
        else:
            self.tr_config = AutoConfig.from_pretrained(
                cfg["model_name"], output_hidden_states=True
            )

        self.tr_config.hidden_dropout = 0.0
        self.tr_config.hidden_dropout_prob = 0.0
        self.tr_config.attention_dropout = 0.0
        self.tr_config.attention_probs_dropout_prob = 0.0

        if cfg["mlm"]:
            self.model = AutoModel.from_pretrained(
                f"../weights/mlm_model/{cfg['model_name']}", config=self.tr_config
            )
        elif cfg["pretrained"]:
            self.model = AutoModel.from_pretrained(
                cfg["model_name"], config=self.tr_config
            )
        else:
            self.model = AutoModel(self.tr_config)
        if self.cfg["transformers_params"]["gradient_checkpointing"]:
            self.model.gradient_checkpointing_enable()
        if cfg["preprocess"]:
            self.model.resize_token_embeddings(len(cfg["tokenizer"]))

        # header
        if cfg["header"] == "cls":
            self.fc = nn.Linear(self.tr_config.hidden_size, 6)
            self._init_weights(self.fc)
        elif cfg["header"] == "mean_pooling":
            self.pool = MeanPooling()
            self.fc = nn.Linear(self.tr_config.hidden_size, 6)
            self._init_weights(self.fc)
        elif cfg["header"] == "attention":
            self.attention = AttentionHead(self.tr_config.hidden_size)
            self.fc = nn.Linear(self.tr_config.hidden_size, 6)
            self._init_weights(self.fc)
        elif cfg["header"] == "weighted_average_pooling":
            layer_start = (
                self.tr_config.num_hidden_layers + 1
            ) - 4  # use last 4-layers
            self.wl_pool = WeightedLayerPooling(
                self.tr_config.num_hidden_layers,
                layer_start=layer_start,
                layer_weights=None,
            )
            self.m_pool = MeanPooling()
            self.fc = nn.Linear(self.tr_config.hidden_size, 6)
            self._init_weights(self.fc)
        elif cfg["header"] == "cls_concatenate":
            self.fc = nn.Linear(self.tr_config.hidden_size * 4, 6)
            self._init_weights(self.fc)
        elif cfg["header"] == "mean_pooling_concatenate":
            self.pool = MeanPooling()
            self.fc = nn.Linear(self.tr_config.hidden_size * 4, 6)
            self._init_weights(self.fc)
        elif cfg["header"] == "1dcnn":
            self.cnn1 = nn.Conv1d(
                self.tr_config.hidden_size, 256, kernel_size=2, padding=1
            )
            self.cnn2 = nn.Conv1d(256, 6, kernel_size=2, padding=1)
        elif cfg["header"] == "lstm":
            self.lstm = nn.LSTM(
                self.tr_config.hidden_size,
                self.tr_config.hidden_size,
                batch_first=True,
            )
            self.fc = nn.Linear(self.tr_config.hidden_size, 6)
            self._init_weights(self.fc)
        elif cfg["header"] == "gru":
            self.gru = nn.GRU(
                self.tr_config.hidden_size,
                self.tr_config.hidden_size,
                batch_first=True,
            )
            self.fc = nn.Linear(self.tr_config.hidden_size, 6)
            self._init_weights(self.fc)

        # reinit same layers
        if cfg["transformers_params"]["reinit_layers"] is not None:
            print(f"reinit {cfg['transformers_params']['reinit_layers']} layers")
            self._reinit_layer(self.model)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.tr_config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.tr_config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def _reinit_layer(self, model):
        if "funnel" in self.cfg["model_name"]:
            for layer in model.decoder.layers[
                -self.cfg["transformers_params"]["reinit_layers"] :
            ]:
                for module in layer.modules():
                    if isinstance(module, nn.Linear):
                        module.weight.data.normal_(
                            mean=0.0, std=model.config.initializer_range
                        )
                        if module.bias is not None:
                            module.bias.data.zero_()
                    elif isinstance(module, nn.Embedding):
                        module.weight.data.normal_(
                            mean=0.0, std=model.config.initializer_range
                        )
                        if module.padding_idx is not None:
                            module.weight.data[module.padding_idx].zero_()
                    elif isinstance(module, nn.LayerNorm):
                        module.bias.data.zero_()
                        module.weight.data.fill_(1.0)
        else:
            for layer in model.encoder.layer[
                -self.cfg["transformers_params"]["reinit_layers"] :
            ]:
                for module in layer.modules():
                    if isinstance(module, nn.Linear):
                        module.weight.data.normal_(
                            mean=0.0, std=model.config.initializer_range
                        )
                        if module.bias is not None:
                            module.bias.data.zero_()
                    elif isinstance(module, nn.Embedding):
                        module.weight.data.normal_(
                            mean=0.0, std=model.config.initializer_range
                        )
                        if module.padding_idx is not None:
                            module.weight.data[module.padding_idx].zero_()
                    elif isinstance(module, nn.LayerNorm):
                        module.bias.data.zero_()
                        module.weight.data.fill_(1.0)

    def forward(self, inputs):
        outputs = self.model(**inputs)
        if self.cfg["header"] == "cls":
            last_hidden_states = outputs[0][:, 0]
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(outputs, p=2, dim=1)
            else:
                outputs = self.fc(last_hidden_states)
        elif self.cfg["header"] == "mean_pooling":
            last_hidden_states = outputs[0]
            feature = self.pool(last_hidden_states, inputs["attention_mask"])
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(feature, p=2, dim=1)
            else:
                outputs = self.fc(feature)
        elif self.cfg["header"] == "attention":
            last_hidden_states = outputs[0]
            feature = self.attention(last_hidden_states)
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(feature, p=2, dim=1)
            else:
                outputs = self.fc(feature)
        elif self.cfg["header"] == "weighted_average_pooling":
            all_hidden_states = torch.stack(outputs["hidden_states"])
            weighted_pooling_embeddings = self.wl_pool(all_hidden_states)
            feature = self.m_pool(weighted_pooling_embeddings, inputs["attention_mask"])
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(feature, p=2, dim=1)
            else:
                outputs = self.fc(feature)
        elif self.cfg["header"] == "cls_concatenate":
            feature = torch.cat(
                [outputs["hidden_states"][-1 * i][:, 0] for i in range(1, 4 + 1)], dim=1
            )
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(feature, p=2, dim=1)
            else:
                outputs = self.fc(feature)
        if self.cfg["header"] == "mean_pooling_concatenate":
            features = []
            for i in range(1, 4 + 1):
                last_hidden_states = outputs["hidden_states"][-1 * i]
                feature = self.pool(last_hidden_states, inputs["attention_mask"])
                features.append(feature)
            feature = torch.cat(features, dim=1)
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(feature, p=2, dim=1)
            else:
                outputs = self.fc(feature)
        elif self.cfg["header"] == "1d_cnn":
            last_hidden_states = outputs["last_hidden_state"].permute(0, 2, 1)
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(last_hidden_states, p=2, dim=1)
            else:
                cnn_embeddings = F.relu(self.cnn1(last_hidden_states))
                cnn_embeddings = self.cnn2(cnn_embeddings)
                outputs, _ = torch.max(cnn_embeddings, 2)
        elif self.cfg["header"] == "lstm":
            feature, _ = self.lstm(outputs["last_hidden_state"], None)
            feature = feature[:, -1, :]
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(feature, p=2, dim=1)
            else:
                outputs = self.fc(feature)
        elif self.cfg["header"] == "gru":
            feature, _ = self.gru(outputs["last_hidden_state"], None)
            feature = feature[:, -1, :]
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(feature, p=2, dim=1)
            else:
                outputs = self.fc(feature)
        return outputs

    def collate(self, inputs):
        # 一番長いtokenへ合わせる
        mask_len = int(inputs["attention_mask"].sum(axis=1).max())
        for k, v in inputs.items():
            inputs[k] = inputs[k][:, :mask_len]
        return inputs

    def training_step(self, batch, batch_idx):
        X, y = batch
        X = self.collate(X)
        if self.cfg["awp"] is not None:
            # awp step
            opt = self.optimizers()
            sch = self.lr_schedulers()

            # with torch.cuda.amp.autocast(enabled=self.cfg["awp"]["amp"]):
            pred_y = self.forward(X)
            loss = self.criterion(pred_y, y)

            if self.awp_accumulate_grad_batches > 1:
                loss = loss / self.awp_accumulate_grad_batches
            # self.awp_scaler.scale(loss).backward()
            self.manual_backward(loss)

            if (batch_idx + 1) % self.awp_accumulate_grad_batches == 0:
                if self.trainer.current_epoch >= self.awp_start_epoch:
                    self._awp_save()
                    for _ in range(self.adv_step):
                        self._awp_attack_step()
                        # with torch.cuda.amp.autocast(enabled=self.cfg["awp"]["amp"]):
                        pred_y = self.forward(X)
                        adv_loss = self.criterion(pred_y, y)
                        opt.zero_grad()
                        # self.awp_scaler.scale(adv_loss).backward()
                        self.manual_backward(adv_loss)
                    self._awp_restore()

                # self.awp_scaler.unscale_(opt)
                # torch.nn.utils.clip_grad_norm_(
                #    self.parameters(), self.awp_max_grad_norm
                # )
                # self.awp_scaler.step(opt)
                opt.step()
                # self.awp_scaler.update()
                opt.zero_grad()
                sch.step()
        else:
            # normal step
            pred_y = self.forward(X)
            loss = self.criterion(pred_y, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def training_epoch_end(self, outputs):
        loss_list = [x["loss"] for x in outputs]
        avg_loss = torch.stack(loss_list).mean()
        self.log("train_avg_loss", avg_loss, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        X, y = batch
        X = self.collate(X)
        # if self.cfg["awp"] is not None:
        #    with torch.cuda.amp.autocast(enabled=self.cfg["awp"]["amp"]):
        #        pred_y = self.forward(X)
        #        loss = self.criterion(pred_y, y)
        # else:
        pred_y = self.forward(X)
        loss = self.criterion(pred_y, y)
        return {"valid_loss": loss, "preds": pred_y, "targets": y}

    def validation_epoch_end(self, outputs):
        loss_list = [x["valid_loss"] for x in outputs]
        preds = torch.cat([x["preds"] for x in outputs], dim=0).cpu().detach().numpy()
        targets = (
            torch.cat([x["targets"] for x in outputs], dim=0).cpu().detach().numpy()
        )
        avg_loss = torch.stack(loss_list).mean()
        score, scores = mcrmse(targets, preds)
        self.log("valid_avg_loss", avg_loss, prog_bar=True)
        self.log("valid_score", score, prog_bar=True)
        self.log("valid_cohesion", scores[0], prog_bar=True)
        self.log("valid_syntax", scores[1], prog_bar=True)
        self.log("valid_vocabulary", scores[2], prog_bar=True)
        self.log("valid_phraseology	", scores[3], prog_bar=True)
        self.log("valid_grammar", scores[4], prog_bar=True)
        self.log("valid_conventions", scores[5], prog_bar=True)
        return avg_loss

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        X, _ = batch
        X = self.collate(X)
        pred_y = self.forward(X)
        return pred_y

    def get_scheduler(self, optimizer, num_train_steps):
        if self.cfg["transformers_params"]["scheduler"] == "linear":
            scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=self.cfg["transformers_params"]["warmup_ratio"]
                * num_train_steps,
                num_training_steps=num_train_steps,
            )
        elif self.cfg["transformers_params"]["scheduler"] == "cosine":
            scheduler = get_cosine_schedule_with_warmup(
                optimizer,
                num_warmup_steps=self.cfg["transformers_params"]["warmup_ratio"]
                * num_train_steps,
                num_training_steps=num_train_steps,
                num_cycles=self.cfg["transformers_params"]["num_cycles"],
            )
        return scheduler

    def configure_optimizers(self):
        # ↓ decayする層を選択 & header (decoder) には別の学習率を設定
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {
                "params": [p for n, p in self.named_parameters() if "model" not in n],
                "lr": self.cfg["transformers_params"]["decoder_lr"],
                "weight_decay": 0.0,
            },
        ]

        if "funnel" in self.cfg["model_name"]:
            layers = (
                [self.model.embeddings]
                + list(self.model.encoder.blocks)
                + list(self.model.decoder.layers)
            )
        else:
            layers = [self.model.embeddings] + list(self.model.encoder.layer)
        layers.reverse()
        lr = self.cfg["transformers_params"]["encoder_lr"]
        lr_decay = self.cfg["transformers_params"]["lr_decay_final"] ** (
            1.0 / len(layers)
        )
        for layer in layers:
            optimizer_parameters += [
                {
                    "params": [
                        p
                        for n, p in layer.named_parameters()
                        if not any(nd in n for nd in no_decay)
                    ],
                    "weight_decay": self.cfg["transformers_params"]["weight_decay"],
                    "lr": lr,
                },
                {
                    "params": [
                        p
                        for n, p in layer.named_parameters()
                        if any(nd in n for nd in no_decay)
                    ],
                    "weight_decay": 0.0,
                    "lr": lr,
                },
            ]
            lr *= lr_decay

        optimizer = optim.AdamW(
            optimizer_parameters, lr=self.cfg["transformers_params"]["encoder_lr"],
        )

        if self.cfg["awp"] is None:
            num_train_steps = self.trainer.estimated_stepping_batches
        else:
            num_train_steps = (
                self.trainer.estimated_stepping_batches
                / self.awp_accumulate_grad_batches
            )

        scheduler = self.get_scheduler(optimizer, num_train_steps)
        scheduler = {"scheduler": scheduler, "interval": "step", "frequency": 1}
        return [optimizer], [scheduler]

    ###############################################################################
    # awp -------------------------------------------------------------------------
    ###############################################################################
    def on_before_optimizer_step(self, optimizer, optimizer_idx):
        if self.cfg["awp"] is not None:
            self.clip_gradients(
                optimizer,
                gradient_clip_val=self.awp_max_grad_norm,
                gradient_clip_algorithm=None,
            )

    def _awp_attack_step(self):
        e = 1e-6
        for name, param in self.named_parameters():
            if (
                param.requires_grad
                and param.grad is not None
                and self.adv_param in name
            ):
                norm1 = torch.norm(param.grad)
                norm2 = torch.norm(param.data.detach())
                if norm1 != 0 and not torch.isnan(norm1):
                    r_at = self.adv_lr * param.grad / (norm1 + e) * (norm2 + e)
                    param.data.add_(r_at)
                    param.data = torch.min(
                        torch.max(param.data, self.backup_eps[name][0]),
                        self.backup_eps[name][1],
                    )
                # param.data.clamp_(*self.backup_eps[name])

    def _awp_save(self):
        for name, param in self.named_parameters():
            if (
                param.requires_grad
                and param.grad is not None
                and self.adv_param in name
            ):
                if name not in self.backup:
                    self.backup[name] = param.data.clone()
                    grad_eps = self.adv_eps * param.abs().detach()
                    self.backup_eps[name] = (
                        self.backup[name] - grad_eps,
                        self.backup[name] + grad_eps,
                    )

    def _awp_restore(self):
        for name, param in self.named_parameters():
            if name in self.backup:
                param.data = self.backup[name]
        self.backup = {}
        self.backup_eps = {}

In [13]:
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, cfg, X, y=None):
        self.cfg = cfg
        if y is None:
            self.X = X.values
            self.y = torch.zeros(len(self.X), dtype=torch.float32)
        else:
            self.X = X.values
            self.y = torch.tensor(y.values, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        X = self._prepare_input(self.X[index])
        y = self.y[index]
        return X, y

    def _prepare_input(self, X):
        if self.cfg["token_cut_head_and_tail"]:
            X = self.cut_head_and_tail(X)
        else:
            X = self.cfg["tokenizer"].encode_plus(
                X,
                return_tensors=None,
                add_special_tokens=True,
                max_length=self.cfg["tokenizer_params"]["max_length"],
                padding="max_length",
                truncation=True,
            )
        for k, v in X.items():
            X[k] = torch.tensor(v, dtype=torch.long)
        return X

    def cut_head_and_tail(self, text):
        # まずは限界を設定せずにトークナイズする
        max_len = self.cfg["tokenizer_params"]["max_length"]
        input_ids = self.cfg["tokenizer"].encode(text)
        n_token = len(input_ids)

        # トークン数が最大数と同じ場合
        if n_token == max_len:
            input_ids = input_ids
            attention_mask = [1 for _ in range(max_len)]
            token_type_ids = [1 for _ in range(max_len)]
        # トークン数が最大数より少ない場合
        elif n_token < max_len:
            pad = [1 for _ in range(max_len - n_token)]
            input_ids = input_ids + pad
            attention_mask = [1 if n_token > i else 0 for i in range(max_len)]
            token_type_ids = [1 if n_token > i else 0 for i in range(max_len)]
        # トークン数が最大数より多い場合
        else:
            harf_len = (max_len - 2) // 2
            _input_ids = input_ids[1:-1]
            input_ids = [0] + _input_ids[:harf_len] + _input_ids[-harf_len:] + [2]
            attention_mask = [1 for _ in range(max_len)]
            token_type_ids = [1 for _ in range(max_len)]

        d = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
        }

        return d

In [14]:
class TransformersRegressorInference:
    def __init__(self, cfg, weight_path=None):
        # tokenizer
        tokenizer = AutoTokenizer.from_pretrained(cfg["model_name"])
        if cfg["preprocess"]:
            tokenizer.add_tokens("[BR]", special_tokens=True)
        cfg["tokenizer"] = tokenizer

        self.model = TransformersModel(cfg)
        self.weight_path = weight_path
        self.cfg = cfg
        self.trainer = Trainer(**self.cfg["pl_params"])

    def predict(self, test_X):
        preds = []
        test_dataset = TableDataset(self.cfg, test_X)
        test_dataloader = torch.utils.data.DataLoader(
            test_dataset, **self.cfg["test_loader"],
        )
        preds = self.trainer.predict(
            self.model, dataloaders=test_dataloader, ckpt_path=self.weight_path
        )
        preds = torch.cat(preds, axis=0)
        preds = preds.cpu().detach().numpy()
        return preds

Data Preparation

In [15]:
# read csv
train = pd.read_csv("../data/input/train.csv")
test = pd.read_csv("../data/input/test.csv")
sub = pd.read_csv("../data/input/sample_submission.csv")

# split X/y
train_X = train["full_text"]
train_y = train.drop(["text_id", "full_text"], axis=1)
test_X = test["full_text"]

inference

In [16]:
def one_fold_valid(skf, cfg, valid_X, valid_y, fold_n):
    print(f"[fold_{fold_n}]")
    seed_everything(cfg["general"]["seed"], workers=True)

    if use_computed:
        valid_preds = joblib.load(f"../data/preds/valid_{cfg['general']['seed']}_{cfg['general']['save_name']}_{fold_n}.preds")
    else:
        model = TransformersRegressorInference(cfg, f"{cfg['save_weight_folder']}/last_epoch_fold{fold_n}.ckpt")
        valid_preds = model.predict(valid_X)
        
        del model
        gc.collect()
        torch.cuda.empty_cache()

    score, scores = mcrmse(valid_y.values, valid_preds)
    #print(f"mcrmse_score:{score}, mcrmse_scores:{scores}")

    return valid_preds

In [17]:
def one_fold_test(cfg, test_X, fold_n):
    print(f"[fold_{fold_n}]")
    seed_everything(cfg["general"]["seed"], workers=True)

    model = TransformersRegressorInference(cfg, f"{cfg['save_weight_folder']}/last_epoch_fold{fold_n}.ckpt")
    test_preds = model.predict(test_X)

    del model
    gc.collect()
    torch.cuda.empty_cache()

    return test_preds

In [18]:
# random seed setting
seed_everything(cfg1["general"]["seed"], workers=True)

Global seed set to 42


42

In [19]:
use_computed = True

optimize weights only

In [20]:
fold_list = range(cfg1["general"]["n_splits"])
weights_list = [[] for _ in range(6)]
skf = MultilabelStratifiedKFold(
    n_splits=cfg1["general"]["n_splits"], shuffle=True, random_state=cfg1["general"]["seed"]
)
for j, fold_n in enumerate(fold_list):
    preds_list = []
    _, valid_indices = list(skf.split(train_X, train_y))[fold_n]
    valid_X_cv, valid_y_cv = (
        train_X.iloc[valid_indices].reset_index(drop=True),
        train_y.iloc[valid_indices].reset_index(drop=True),
    )

    for i, cfg in enumerate(cfg_list):
        valid_preds = one_fold_valid(skf, cfg, valid_X_cv, valid_y_cv, fold_n)
        preds_list.append(valid_preds)
        del valid_preds
        gc.collect()

    score, scores = mcrmse(valid_y_cv.values, np.mean(preds_list, axis=0))
    print()
    print(f"simple_mean: mcrmse_score:{score}, mcrmse_scores:{scores}")
    for i, p in enumerate(preds_list):
        score, scores = mcrmse(valid_y_cv.values, p)
        print(f"cfg_{i}: mcrmse_score:{score}, mcrmse_scores:{scores}")

    preds_list = np.array(preds_list)
    cohesion = preds_list[:, :, 0]
    syntax = preds_list[:, :, 1]
    vocabulary = preds_list[:, :, 2]
    phraseology = preds_list[:, :, 3]
    grammar = preds_list[:, :, 4]
    conventions = preds_list[:, :, 5]
    target_list = [cohesion, syntax, vocabulary, phraseology, grammar, conventions]
    
    for t_idx, target in enumerate(target_list):

        def f(x):
            pred = np.zeros_like(target[0])
            for i, p in enumerate(target):
                pred += p * x[i]
            score = metrics.mean_squared_error(
                valid_y_cv.values[:, t_idx], pred, squared=False
            )
            return score

        init_state = np.ones((len(target))) / len(target)
        bounds = [(0.0, 1.0)] * len(target)
        result = minimize(f, init_state, method="Nelder-Mead", bounds=bounds)
        print(f"optimized_corr:{result['fun']}")

        weights = [[0] for _ in range(len(target))]
        for i in range(len(target)):
            weights[i] = result["x"][i]
        weights_list[t_idx].append(weights)
        print(f"weights:{weights}")

avg_weights_cohesion = np.mean(weights_list[0], axis=0)
avg_weights_syntax = np.mean(weights_list[1], axis=0)
avg_weights_vocabulary = np.mean(weights_list[2], axis=0)
avg_weights_phraseology = np.mean(weights_list[3], axis=0)
avg_weights_grammar = np.mean(weights_list[4], axis=0)
avg_weights_conventions = np.mean(weights_list[5], axis=0)

print()
print(f"cohesion averaged_weights:{avg_weights_cohesion}")
print(f"syntax averaged_weights:{avg_weights_syntax}")
print(f"vocabulary averaged_weights:{avg_weights_vocabulary}")
print(f"phraseology averaged_weights:{avg_weights_phraseology}")
print(f"grammar averaged_weights:{avg_weights_grammar}")
print(f"conventions averaged_weights:{avg_weights_conventions}")

Global seed set to 42


[fold_0]


Global seed set to 42


[fold_0]
[fold_0]


Global seed set to 42
Global seed set to 42


...

[fold_4]
[fold_4]


Global seed set to 42
Global seed set to 42


[fold_4]
[fold_4]


Global seed set to 42


[fold_4]

simple_mean: mcrmse_score:0.4418603842045263, mcrmse_scores:[0.4650295479620753, 0.4221627827221631, 0.4122575878243325, 0.4502636911368272, 0.462159029038597, 0.43928966654316226]
cfg_0: mcrmse_score:0.4517432058281796, mcrmse_scores:[0.47790252081984874, 0.42759786637027475, 0.4232340058467752, 0.4588235932990546, 0.47044895824130306, 0.45245229039182144]
cfg_1: mcrmse_score:0.4474640998250639, mcrmse_scores:[0.47378012786355067, 0.42520154386126247, 0.41987411796431484, 0.4547621280443761, 0.4684081530656485, 0.44275852815123135]
cfg_2: mcrmse_score:0.45474245588007145, mcrmse_scores:[0.47896108741292504, 0.4353564752441213, 0.419943114849705, 0.46247888795965925, 0.47652757759760483, 0.4551875922164135]
cfg_3: mcrmse_score:0.4509786768557819, mcrmse_scores:[0.4730383834428002, 0.43105428880088453, 0.420076257551725, 0.45826358489383207, 0.47439672919999937, 0.44904281724545003]
cfg_4: mcrmse_score:0.4456061886395044, mcrmse_scores:[0.47043601950456565, 0.4247738142780557,

In [21]:
for cfg, w in zip(cfg_list, np.mean(weights_list, axis=(0,1))):
    print(f"{cfg['save_weight_folder'][9:]}: mean_weight -> {w}")

ghts/deberta_v3_base_4096_reinit: mean_weight -> 0.0639014854117706
ghts/deberta_large_512_reinit: mean_weight -> 0.09222912608366358
ghts/muppet_large_512_reinit: mean_weight -> 0.08918373059037628
ghts/deberta_xlarge_512_reinit: mean_weight -> 0.07966986128643644
ghts/deberta_v3_base_512_reinit_pseudo: mean_weight -> 0.041184189963520794
ghts/deberta_v3_large_4096_reinit_pseudo: mean_weight -> 0.05442054351642984
ghts/deberta_xlarge_512_reinit_pseudo: mean_weight -> 0.09187152607213299
ghts/deberta_v3_base_512_reinit_svr_pseudo: mean_weight -> 0.08818349012236444
ghts/deberta_v3_large_4096_reinit_svr_pseudo: mean_weight -> 0.0713517514210638
ghts/deberta_xlarge_512_reinit_svr_pseudo: mean_weight -> 0.09428305593029931
ghts/deberta_v3_large_4096_svr: mean_weight -> 0.09237627122236201
ghts/deberta_large_512_svr: mean_weight -> 0.05250456537893666
ghts/deberta_v3_base_4096_svr: mean_weight -> 0.054920048089673316


KeyError: 'save_weight_folder'

In [22]:
for cfg, w in zip(cfg_list, np.mean(weights_list, axis=(0,1))):
    print(f"{cfg['save_weight_folder'][9:]}: mean_weight -> {w}")

ghts/deberta_v3_base_512_reinit: mean_weight -> 0.01808425191965541
ghts/deberta_v3_base_4096_reinit: mean_weight -> 0.07036672875689522
ghts/deberta_large_512_reinit: mean_weight -> 0.13240250115054272
ghts/muppet_large_512_reinit: mean_weight -> 0.11607240887900794
ghts/deberta_xlarge_512_reinit: mean_weight -> 0.1378779334227738
ghts/deberta_v3_base_512_reinit_pseudo: mean_weight -> 0.06254180381731195
ghts/deberta_v3_large_512_reinit_pseudo: mean_weight -> 0.11073394898863492
ghts/deberta_v3_large_4096_reinit_pseudo: mean_weight -> 0.10644554013071185
ghts/deberta_xlarge_512_reinit_pseudo: mean_weight -> 0.14485640234187258
ghts/deberta_v3_base_512_reinit_svr_pseudo: mean_weight -> 0.10178945156293184


In [16]:
for cfg, w in zip(cfg_list, np.mean(weights_list, axis=(0,1))):
    print(f"{cfg['save_weight_folder'][9:]}: mean_weight -> {w}")

ghts/deberta_v3_base_512_reinit_pseudo: mean_weight -> 0.11638717828936428
ghts/deberta_v3_base_4096_reinit_pseudo: mean_weight -> 0.11545940443971185
ghts/deberta_v3_large_512_reinit_pseudo: mean_weight -> 0.12636836477094585
ghts/deberta_v3_large_4096_reinit_pseudo: mean_weight -> 0.11533129744016073
ghts/deberta_large_512_reinit_pseudo: mean_weight -> 0.12631688386954354
ghts/muppet_large_512_reinit_pseudo: mean_weight -> 0.1146294283697651
ghts/deberta_xlarge_512_reinit_pseudo: mean_weight -> 0.28604372202085193


In [28]:
for cfg, w in zip(cfg_list, np.mean(weights_list, axis=(0,1))):
    print(f"{cfg['save_weight_folder'][9:]}: mean_weight -> {w}")

ghts/deberta_v3_base_512_reinit: mean_weight -> 0.0851821266004092
ghts/deberta_v3_base_4096_reinit: mean_weight -> 0.15075516345919657
ghts/deberta_v3_large_512_reinit: mean_weight -> 0.1153585078458238
ghts/deberta_v3_large_4096_reinit: mean_weight -> 0.09972915365392339
ghts/deberta_large_512_reinit: mean_weight -> 0.1261354354129243
ghts/muppet_large_512_reinit: mean_weight -> 0.18652713693522452
ghts/deberta_xlarge_512_reinit: mean_weight -> 0.231936107009877


blending only

In [23]:
preds_list_valid_cv = [[] for i in range(cfg1["general"]["n_splits"])]
score_cv = []
scores_cv = []
simple_score_cv = []
simple_scores_cv = []
for j, fold_n in enumerate(fold_list):
    preds_list = []
    _, valid_indices = list(skf.split(train_X, train_y))[fold_n]
    valid_X_cv, valid_y_cv = (
        train_X.iloc[valid_indices].reset_index(drop=True),
        train_y.iloc[valid_indices].reset_index(drop=True),
    )

    for i, cfg in enumerate(cfg_list):
        valid_preds = one_fold_valid(skf, cfg, valid_X_cv, valid_y_cv, fold_n)
        preds_list.append(valid_preds)
        del valid_preds
        gc.collect()

    score, scores = mcrmse(valid_y_cv.values, np.mean(preds_list, axis=0))
    print()
    print(f"simple_mean: mcrmse_score:{score}, mcrmse_scores:{scores}")
    simple_score_cv.append(score)
    simple_scores_cv.append(scores)
    for i, p in enumerate(preds_list):
        score, scores = mcrmse(valid_y_cv.values, p)
        print(f"cfg_{i}: mcrmse_score:{score}, mcrmse_scores:{scores}")

    preds_list = np.array(preds_list)
    
    # blending
    preds_list_valid_cv[j] = np.zeros(preds_list[0].shape)
    for i in range(preds_list.shape[0]):
        preds_list_valid_cv[j][:, 0] += preds_list[i, :, 0] * avg_weights_cohesion[i]
        preds_list_valid_cv[j][:, 1] += preds_list[i, :, 1] * avg_weights_syntax[i]
        preds_list_valid_cv[j][:, 2] += preds_list[i, :, 2] * avg_weights_vocabulary[i]
        preds_list_valid_cv[j][:, 3] += preds_list[i, :, 3] * avg_weights_phraseology[i]
        preds_list_valid_cv[j][:, 4] += preds_list[i, :, 4] * avg_weights_grammar[i]
        preds_list_valid_cv[j][:, 5] += preds_list[i, :, 5] * avg_weights_conventions[i]
        
    score, scores = mcrmse(valid_y_cv.values, preds_list_valid_cv[j])
    print(f"optimized: mcrmse_score:{score}, mcrmse_scores:{scores}")
    score_cv.append(score)
    scores_cv.append(scores)

print(f"simple mean_cv score:{np.mean(simple_score_cv)}")
print(f"simple mean_cv score:{np.mean(simple_scores_cv, axis=0)}")
print(f"optimized mean_cv score:{np.mean(score_cv)}")
print(f"optimized mean_cv score:{np.mean(scores_cv, axis=0)}")
for i in range(len(score_cv)):
    print(f"fold{i}: score_cv[i]")

Global seed set to 42


[fold_0]


Global seed set to 42


[fold_0]


Global seed set to 42


...

[fold_4]
[fold_4]


Global seed set to 42
Global seed set to 42


[fold_4]
[fold_4]


Global seed set to 42



simple_mean: mcrmse_score:0.4418603842045263, mcrmse_scores:[0.4650295479620753, 0.4221627827221631, 0.4122575878243325, 0.4502636911368272, 0.462159029038597, 0.43928966654316226]
cfg_0: mcrmse_score:0.4517432058281796, mcrmse_scores:[0.47790252081984874, 0.42759786637027475, 0.4232340058467752, 0.4588235932990546, 0.47044895824130306, 0.45245229039182144]
cfg_1: mcrmse_score:0.4474640998250639, mcrmse_scores:[0.47378012786355067, 0.42520154386126247, 0.41987411796431484, 0.4547621280443761, 0.4684081530656485, 0.44275852815123135]
cfg_2: mcrmse_score:0.45474245588007145, mcrmse_scores:[0.47896108741292504, 0.4353564752441213, 0.419943114849705, 0.46247888795965925, 0.47652757759760483, 0.4551875922164135]
cfg_3: mcrmse_score:0.4509786768557819, mcrmse_scores:[0.4730383834428002, 0.43105428880088453, 0.420076257551725, 0.45826358489383207, 0.47439672919999937, 0.44904281724545003]
cfg_4: mcrmse_score:0.4456061886395044, mcrmse_scores:[0.47043601950456565, 0.4247738142780557, 0.417271

In [24]:
print(f"optimized mean_cv score:{np.mean(score_cv)}")
print(f"optimized mean_cv score:{np.mean(scores_cv, axis=0)}")
for i in range(len(score_cv)):
    print(f"fold{i}: {score_cv[i]}")

optimized mean_cv score:0.44362870251316283
optimized mean_cv score:[0.47044204 0.43718095 0.40666706 0.44733399 0.46264106 0.43750711]
fold0: 0.43938150040185214
fold1: 0.4443667928148725
fold2: 0.44862425316992405
fold3: 0.444449214617516
fold4: 0.4413217515616495


In [32]:
print(f"optimized mean_cv score:{np.mean(score_cv)}")
print(f"optimized mean_cv score:{np.mean(scores_cv, axis=0)}")
for i in range(len(score_cv)):
    print(f"fold{i}: {score_cv[i]}")

optimized mean_cv score:0.4442963443345695
optimized mean_cv score:[0.47144265 0.43754408 0.40820216 0.44737376 0.46324763 0.43796779]
fold0: 0.43962936055108504
fold1: 0.4446806390236411
fold2: 0.4495163950172176
fold3: 0.44551959500246796
fold4: 0.4421357320784356


In [18]:
print(f"optimized mean_cv score:{np.mean(score_cv)}")
print(f"optimized mean_cv score:{np.mean(scores_cv, axis=0)}")
for i in range(len(score_cv)):
    print(f"fold{i}: {score_cv[i]}")

optimized mean_cv score:0.4439758496123359
optimized mean_cv score:[0.47048388 0.43723195 0.40779752 0.44744441 0.46317166 0.43772568]
fold0: 0.43966736498166054
fold1: 0.4447260160321184
fold2: 0.4490585447406838
fold3: 0.44465007540737683
fold4: 0.44177724689983977


In [32]:
print(f"optimized mean_cv score:{np.mean(score_cv)}")
print(f"optimized mean_cv score:{np.mean(scores_cv, axis=0)}")
for i in range(len(score_cv)):
    print(f"fold{i}: {score_cv[i]}")

optimized mean_cv score:0.4435757923511847
optimized mean_cv score:[0.47044186 0.43698109 0.40659112 0.44745059 0.4626017  0.43738839]
fold0: 0.4392709463063429
fold1: 0.44423089670219973
fold2: 0.44890168720608276
fold3: 0.4442636474010416
fold4: 0.4412117841402565


In [26]:
print(f"optimized mean_cv score:{np.mean(score_cv)}")
print(f"optimized mean_cv score:{np.mean(scores_cv, axis=0)}")

optimized mean_cv score:0.44402996448356824
optimized mean_cv score:[0.47047417 0.43735069 0.40790291 0.44751299 0.46322585 0.43771317]


In [17]:
print(f"optimized mean_cv score:{np.mean(score_cv)}")
print(f"optimized mean_cv score:{np.mean(scores_cv, axis=0)}")

optimized mean_cv score:0.444213507082441
optimized mean_cv score:[0.47145325 0.43764378 0.40772634 0.44727526 0.463229   0.4379534 ]


In [23]:
print(f"optimized mean_cv score:{np.mean(score_cv)}")
print(f"optimized mean_cv score:{np.mean(scores_cv, axis=0)}")

optimized mean_cv score:0.4442963443345695
optimized mean_cv score:[0.47144265 0.43754408 0.40820216 0.44737376 0.46324763 0.43796779]


In [50]:
print(f"optimized mean_cv score:{np.mean(score_cv)}")
print(f"optimized mean_cv score:{np.mean(scores_cv, axis=0)}")

optimized mean_cv score:0.4444073891630994
optimized mean_cv score:[0.4716746  0.43770278 0.40836213 0.44728157 0.46330418 0.43811907]


optimize blending only (ridge)

In [14]:
def one_fold_train_valid(skf, cfg, train_X, valid_X, valid_y, fold_n):
    print(f"[fold_{fold_n}]")
    seed_everything(cfg["general"]["seed"], workers=True)

    if use_computed:
        valid_preds = joblib.load(f"../data/preds/valid_{cfg['general']['seed']}_{cfg['general']['save_name']}_{fold_n}.preds")
        train_preds = joblib.load(f"../data/preds/train_{cfg['general']['seed']}_{cfg['general']['save_name']}_{fold_n}.preds")
    else:
        model = TransformersRegressorInference(cfg, f"{cfg['save_weight_folder']}/last_epoch_fold{fold_n}.ckpt")
        train_preds = model.predict(train_X)
        valid_preds = model.predict(valid_X)
        
        del model
        gc.collect()
        torch.cuda.empty_cache()

    score, scores = mcrmse(valid_y.values, valid_preds)
    #print(f"mcrmse_score:{score}, mcrmse_scores:{scores}")

    return train_preds, valid_preds

In [17]:
fold_list = range(cfg1["general"]["n_splits"])
ridge_list = [[] for _ in range(6)]
skf = MultilabelStratifiedKFold(
    n_splits=cfg1["general"]["n_splits"], shuffle=True, random_state=cfg1["general"]["seed"]
)
for j, fold_n in enumerate(fold_list):
    preds_list_train = []
    preds_list_valid = []
    train_indices, valid_indices = list(skf.split(train_X, train_y))[fold_n]
    train_X_cv, train_y_cv = (
        train_X.iloc[train_indices].reset_index(drop=True),
        train_y.iloc[train_indices].reset_index(drop=True),
    )
    valid_X_cv, valid_y_cv = (
        train_X.iloc[valid_indices].reset_index(drop=True),
        train_y.iloc[valid_indices].reset_index(drop=True),
    )

    for i, cfg in enumerate(cfg_list):
        train_preds, valid_preds = one_fold_train_valid(skf, cfg, train_X_cv, valid_X_cv, valid_y_cv, fold_n)
        preds_list_train.append(train_preds)
        preds_list_valid.append(valid_preds)
        del valid_preds, train_preds
        gc.collect()

    score, scores = mcrmse(valid_y_cv.values, np.mean(preds_list_valid, axis=0))
    print()
    print(f"simple_mean: mcrmse_score:{score}, mcrmse_scores:{scores}")
    for i, p in enumerate(preds_list_valid):
        score, scores = mcrmse(valid_y_cv.values, p)
        print(f"cfg_{i}: mcrmse_score:{score}, mcrmse_scores:{scores}")

    preds_list_train = np.array(preds_list_train)
    cohesion_train = preds_list_train[:, :, 0]
    syntax_train = preds_list_train[:, :, 1]
    vocabulary_train = preds_list_train[:, :, 2]
    phraseology_train = preds_list_train[:, :, 3]
    grammar_train = preds_list_train[:, :, 4]
    conventions_train = preds_list_train[:, :, 5]
    target_list_train = [cohesion_train, syntax_train, vocabulary_train, phraseology_train, grammar_train, conventions_train]

    preds_list_valid = np.array(preds_list_valid)
    cohesion_valid = preds_list_valid[:, :, 0]
    syntax_valid = preds_list_valid[:, :, 1]
    vocabulary_valid = preds_list_valid[:, :, 2]
    phraseology_valid = preds_list_valid[:, :, 3]
    grammar_valid = preds_list_valid[:, :, 4]
    conventions_valid = preds_list_valid[:, :, 5]
    target_list_valid = [cohesion_valid, syntax_valid, vocabulary_valid, phraseology_valid, grammar_valid, conventions_valid]
    
    for j, (target_train, target_valid) in enumerate(zip(target_list_train, target_list_valid)):
        #clf = linear_model.BayesianRidge()
        clf = linear_model.Lasso()
        clf.fit(target_train.T, train_y_cv.iloc[:, j])
        ridge_preds = clf.predict(target_valid.T)
        score = metrics.mean_squared_error(
            valid_y_cv.values[:, j], ridge_preds, squared=False
        )
        print(f"mcrmse_score:{score}")
        ridge_list.append(clf)

Global seed set to 42
Global seed set to 42


[fold_0]
[fold_0]


Global seed set to 42



simple_mean: mcrmse_score:0.4426928168821353, mcrmse_scores:[0.4723965305254995, 0.4514796506367955, 0.39961054506159743, 0.4468851391595757, 0.45096718197024305, 0.4348178539391007]
cfg_0: mcrmse_score:0.44501512108121055, mcrmse_scores:[0.4764805445068816, 0.4555338394273924, 0.4005444619125103, 0.44667452673163766, 0.45393005510740025, 0.43692729880144127]
cfg_1: mcrmse_score:0.44585454507407873, mcrmse_scores:[0.4752488045513762, 0.4521792429120024, 0.40275357229592157, 0.4525966114288518, 0.45402999895842294, 0.4383190402978972]
mcrmse_score:0.6487136152041301
mcrmse_score:0.6364518325892338
mcrmse_score:0.5732140998715254
mcrmse_score:0.645201856731233
mcrmse_score:0.6846099611419565
mcrmse_score:0.6755270957628826
[fold_1]


Global seed set to 42


...

[fold_3]


Global seed set to 42



simple_mean: mcrmse_score:0.4469928527020628, mcrmse_scores:[0.46862523782535, 0.4345103312348706, 0.4147866410704356, 0.44777399182239497, 0.4850766082227587, 0.43118430603656693]
cfg_0: mcrmse_score:0.4506304738031212, mcrmse_scores:[0.4747445894753775, 0.4394010368201213, 0.4167011940470564, 0.4509580945825779, 0.48596721450075964, 0.4360107133928339]
cfg_1: mcrmse_score:0.4486714258970248, mcrmse_scores:[0.468638873874748, 0.43482953921371054, 0.41666864833124034, 0.449739106921979, 0.4900074273148525, 0.43214495972561845]
mcrmse_score:0.6712343221487528
mcrmse_score:0.6403512485644408
mcrmse_score:0.6015497522944143
mcrmse_score:0.6622294411559594
mcrmse_score:0.7137995398199728
mcrmse_score:0.6586587521469858
[fold_4]


Global seed set to 42


[fold_4]

simple_mean: mcrmse_score:0.44403544140215906, mcrmse_scores:[0.46905100019013596, 0.42279077971961193, 0.41437917001723124, 0.4513870990230649, 0.46560198363384536, 0.4410026158290654]
cfg_0: mcrmse_score:0.4479013443036936, mcrmse_scores:[0.4718749347060132, 0.4276361751180047, 0.41820788229303213, 0.4569683701660854, 0.46699552890261825, 0.44572517463640804]
cfg_1: mcrmse_score:0.44604118662149134, mcrmse_scores:[0.47309903345380455, 0.4241423030465027, 0.41476055688393076, 0.45127996892811634, 0.4706398927727104, 0.44232536464388345]
mcrmse_score:0.6575495997571503
mcrmse_score:0.6501007785695101
mcrmse_score:0.5891370899821976
mcrmse_score:0.6743956297334848
mcrmse_score:0.7169897558835701
mcrmse_score:0.6697963082165664


optimize and blending

In [22]:
fold_list = range(cfg1["general"]["n_splits"])
preds_list_cv = [[] for i in range(cfg1["general"]["n_splits"])]
weights_list = [[] for _ in range(6)]
skf = MultilabelStratifiedKFold(
    n_splits=cfg1["general"]["n_splits"], shuffle=True, random_state=cfg1["general"]["seed"]
)
for j, fold_n in enumerate(fold_list):
    preds_list = []
    _, valid_indices = list(skf.split(train_X, train_y))[fold_n]
    valid_X_cv, valid_y_cv = (
        train_X.iloc[valid_indices].reset_index(drop=True),
        train_y.iloc[valid_indices].reset_index(drop=True),
    )

    for i, cfg in enumerate(cfg_list):
        valid_preds = one_fold_valid(skf, cfg, valid_X_cv, valid_y_cv, fold_n)
        preds_list.append(valid_preds)
        del valid_preds
        gc.collect()

    score, scores = mcrmse(valid_y_cv.values, np.mean(preds_list, axis=0))
    print()
    print(f"simple_mean: mcrmse_score:{score}, mcrmse_scores:{scores}")
    for i, p in enumerate(preds_list):
        score, scores = mcrmse(valid_y_cv.values, p)
        print(f"cfg_{i}: mcrmse_score:{score}, mcrmse_scores:{scores}")

    preds_list = np.array(preds_list)
    preds_list_cv[j] = preds_list
    cohesion = preds_list[:, :, 0]
    syntax = preds_list[:, :, 1]
    vocabulary = preds_list[:, :, 2]
    phraseology = preds_list[:, :, 3]
    grammar = preds_list[:, :, 4]
    conventions = preds_list[:, :, 5]
    target_list = [cohesion, syntax, vocabulary, phraseology, grammar, conventions]

    for t_idx, target in enumerate(target_list):

        def f(x):
            pred = np.zeros_like(target[0])
            for i, p in enumerate(target):
                pred += p * x[i]
            score = metrics.mean_squared_error(
                valid_y_cv.values[:, t_idx], pred, squared=False
            )
            return score

        init_state = np.ones((len(target))) / len(target)
        bounds = [(0.0, 1.0)] * len(target)
        result = minimize(f, init_state, method="Nelder-Mead", bounds=bounds)
        print(f"optimized_corr:{result['fun']}")

        weights = [[0] for _ in range(len(target))]
        for i in range(len(target)):
            weights[i] = result["x"][i]
        weights_list[t_idx].append(weights)
        print(f"weights:{weights}")

avg_weights_cohesion = np.mean(weights_list[0], axis=0)
avg_weights_syntax = np.mean(weights_list[1], axis=0)
avg_weights_vocabulary = np.mean(weights_list[2], axis=0)
avg_weights_phraseology = np.mean(weights_list[3], axis=0)
avg_weights_grammar = np.mean(weights_list[4], axis=0)
avg_weights_conventions = np.mean(weights_list[5], axis=0)

print()
print(f"cohesion averaged_weights:{avg_weights_cohesion}")
print(f"syntax averaged_weights:{avg_weights_syntax}")
print(f"vocabulary averaged_weights:{avg_weights_vocabulary}")
print(f"phraseology averaged_weights:{avg_weights_phraseology}")
print(f"grammar averaged_weights:{avg_weights_grammar}")
print(f"conventions averaged_weights:{avg_weights_conventions}")

Global seed set to 42


[fold_0]


Global seed set to 42


[fold_0]
[fold_0]


Global seed set to 42
Global seed set to 42


...

Global seed set to 42
Global seed set to 42


[fold_1]
[fold_1]


Global seed set to 42
Global seed set to 42
Global seed set to 42


[fold_1]
[fold_1]


KeyboardInterrupt: 

In [13]:
score_cv = []
scores_cv = []
preds_list_valid_cv = [[] for i in range(cfg1["general"]["n_splits"])]
for j, preds in enumerate(preds_list_cv):

    _, valid_indices = list(skf.split(train_X, train_y))[j]
    valid_X_cv, valid_y_cv = (
        train_X.iloc[valid_indices].reset_index(drop=True),
        train_y.iloc[valid_indices].reset_index(drop=True),
    )
    
    # blending
    preds_list_valid_cv[j] = np.zeros(preds[0].shape)
    for i in range(preds.shape[0]):
        preds_list_valid_cv[j][:, 0] += preds[i, :, 0] * avg_weights_cohesion[i]
        preds_list_valid_cv[j][:, 1] += preds[i, :, 1] * avg_weights_syntax[i]
        preds_list_valid_cv[j][:, 2] += preds[i, :, 2] * avg_weights_vocabulary[i]
        preds_list_valid_cv[j][:, 3] += preds[i, :, 3] * avg_weights_phraseology[i]
        preds_list_valid_cv[j][:, 4] += preds[i, :, 4] * avg_weights_grammar[i]
        preds_list_valid_cv[j][:, 5] += preds[i, :, 5] * avg_weights_conventions[i]
        
    score, scores = mcrmse(valid_y_cv.values, np.mean(preds, axis=0))
    print()
    print(f"simple_mean: mcrmse_score:{score}, mcrmse_scores:{scores}")
    for i, p in enumerate(preds):
        score, scores = mcrmse(valid_y_cv.values, p)
        print(f"cfg_{i}: mcrmse_score:{score}, mcrmse_scores:{scores}")
        
    score, scores = mcrmse(valid_y_cv.values, preds_list_valid_cv[j])
    print(f"optimized: mcrmse_score:{score}, mcrmse_scores:{scores}")
    score_cv.append(score)
    scores_cv.append(scores)

print()
print(f"optimized mean_cv score:{np.mean(score_cv)}")
print(f"optimized mean_cv score:{np.mean(scores_cv, axis=0)}")


simple_mean: mcrmse_score:0.44092151174713784, mcrmse_scores:[0.4720510272664117, 0.4493186587571681, 0.3968925323594917, 0.44337967344423296, 0.44863675854379725, 0.43525042011172543]
cfg_0: mcrmse_score:0.44661679275644395, mcrmse_scores:[0.478136184481341, 0.4589547473544348, 0.4017579316964105, 0.44773613966828874, 0.45087779693826036, 0.44223795639992797]
cfg_1: mcrmse_score:0.4464048391957691, mcrmse_scores:[0.4773659991581123, 0.45743327359847313, 0.4035178521296586, 0.44723533562423085, 0.4525589148944916, 0.44031765976964804]
cfg_2: mcrmse_score:0.44448586643844323, mcrmse_scores:[0.47648330647205905, 0.4505310497008678, 0.40269257694686345, 0.4471697352664887, 0.44927198718123534, 0.4407665430631452]
cfg_3: mcrmse_score:0.44472671646098233, mcrmse_scores:[0.47735512111121164, 0.4516886550344083, 0.40180245438441015, 0.44781363658293616, 0.4508851972175929, 0.4388152344353346]
cfg_4: mcrmse_score:0.45587919658355097, mcrmse_scores:[0.49115707754281196, 0.46586656465627796, 0.

In [14]:
for cfg, w in zip(cfg_list, np.mean(weights_list, axis=(0,1))):
    print(f"{cfg['save_weight_folder'][9:]}: mean_weight -> {w}")

deberta-v3-base-512-reinit: mean_weight -> 0.08815145125656429
deberta-v3-base-4096-reinit: mean_weight -> 0.09672884018322093
deberta-v3-large-512-reinit: mean_weight -> 0.06307039726581137
deberta-v3-large-4096-reinit: mean_weight -> 0.08082699843144515
bigbird-roberta-large-4096-reinit: mean_weight -> 0.06029814640283086
deberta-large-512-reinit: mean_weight -> 0.15154171542860648
electra-large-512-reinit: mean_weight -> 0.06399589048253156
longformer-base-4096-reinit: mean_weight -> 0.015908246062988483
muppet-large-512-reinit: mean_weight -> 0.13418874247733906
roberta-large-512-reinit: mean_weight -> 0.04655535988213684
xlm-roberta-large-512-reinit: mean_weight -> 0.020489615270127694
deberta-xlarge-512-reinit: mean_weight -> 0.17961988879406493


blending test

In [45]:
preds_list_test_cv = [[] for i in range(cfg1["general"]["n_splits"])]
score_cv = []
scores_cv = []
for j, fold_n in enumerate(fold_list):
    preds_list = []

    for i, cfg in enumerate(cfg_list):
        test_preds = one_fold_test(cfg, test_X, fold_n)
        preds_list.append(test_preds)
        del test_preds
        gc.collect()

    preds_list = np.array(preds_list)
    
    # blending
    preds_list_test_cv[j] = np.zeros(preds_list[0].shape)
    for i in range(preds_list.shape[0]):
        preds_list_test_cv[j][:, 0] += preds_list[i, :, 0] * avg_weights_cohesion[i]
        preds_list_test_cv[j][:, 1] += preds_list[i, :, 1] * avg_weights_syntax[i]
        preds_list_test_cv[j][:, 2] += preds_list[i, :, 2] * avg_weights_vocabulary[i]
        preds_list_test_cv[j][:, 3] += preds_list[i, :, 3] * avg_weights_phraseology[i]
        preds_list_test_cv[j][:, 4] += preds_list[i, :, 4] * avg_weights_grammar[i]
        preds_list_test_cv[j][:, 5] += preds_list[i, :, 5] * avg_weights_conventions[i]

[fold_0]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at ../input/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifie

reinit 1 layers


Predicting: 0it [00:00, ?it/s]

[fold_0]


...

Predicting: 0it [00:00, ?it/s]

[fold_4]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at ../input/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT ex

reinit 1 layers


Predicting: 0it [00:00, ?it/s]

In [52]:
final_test_preds = np.mean(preds_list_test_cv, axis=0)

In [53]:
final_test_preds = np.clip(final_test_preds, 1, 5)

In [54]:
sub["cohesion"] = final_test_preds[:, 0]
sub["syntax"] = final_test_preds[:, 1]
sub["vocabulary"] = final_test_preds[:, 2]
sub["phraseology"] = final_test_preds[:, 3]
sub["grammar"] = final_test_preds[:, 4]
sub["conventions"] = final_test_preds[:, 5]

In [55]:
sub

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2.883897,2.775009,3.097567,2.973784,2.719742,2.674564
1,000BAD50D026,2.629768,2.409478,2.743813,2.381966,2.135197,2.639184
2,00367BB2546B,3.564441,3.381812,3.616520,3.518507,3.433326,3.293942


In [ ]:
sub.to_csv(f"submission.csv", index=False)

generate pseudo label

In [12]:
# read external data
ex = pd.read_csv("../data/input/past_fb_data.csv")

ex_X = ex["full_text"]

In [13]:
def one_fold_pseudo(cfg, ex_X, fold_n):
    print(f"[fold_{fold_n}]")
    seed_everything(cfg["general"]["seed"], workers=True)

    model = TransformersRegressorInference(cfg, f"{cfg['save_weight_folder']}/last_epoch_fold{fold_n}.ckpt")
    pseudo_preds = model.predict(ex_X)

    del model
    gc.collect()
    torch.cuda.empty_cache()

    return pseudo_preds

In [14]:
avg_weights_cohesion = [0.05193772, 0.11036315, 0.01717836, 0.06184517, 0.24583778, 0.2668852, 0.2475308]
avg_weights_syntax = [0.15986115, 0.05737773, 0.08191739, 0.06150111, 0.26768993, 0.06645094, 0.30885836]
avg_weights_vocabulary = [0.0599905, 0.06431446, 0.11436567, 0.08179424, 0.09350644, 0.19307011, 0.39327719]
avg_weights_phraseology = [0.04839435, 0.15866904, 0.22683944, 0.08257008, 0.2055661, 0.15263263, 0.12525981]
avg_weights_grammar = [0.13787544, 0.21311838, 0.09743774, 0.08402591, 0.12684375, 0.11145722, 0.23145577]
avg_weights_conventions = [0.05605593, 0.10720015, 0.11871781, 0.14182469, 0.21086025, 0.14346338, 0.22574695]

In [ ]:
fold_list = [3, 4]
pseudo_list_cv = [[] for i in range(cfg1["general"]["n_splits"])]
for fold_n in fold_list:
    preds_list = []
    for i, cfg in enumerate(cfg_list):
        pseudo_preds = one_fold_pseudo(cfg, ex_X, fold_n)
        preds_list.append(pseudo_preds)
        print(pseudo_preds[:10])
        del pseudo_preds
        gc.collect()
        
    preds_list = np.array(preds_list)
    
    # blending
    pseudo_list_cv[fold_n] = np.zeros(preds_list[0].shape)
    for i in range(preds_list.shape[0]):
        pseudo_list_cv[fold_n][:, 0] += preds_list[i, :, 0] * avg_weights_cohesion[i]
        pseudo_list_cv[fold_n][:, 1] += preds_list[i, :, 1] * avg_weights_syntax[i]
        pseudo_list_cv[fold_n][:, 2] += preds_list[i, :, 2] * avg_weights_vocabulary[i]
        pseudo_list_cv[fold_n][:, 3] += preds_list[i, :, 3] * avg_weights_phraseology[i]
        pseudo_list_cv[fold_n][:, 4] += preds_list[i, :, 4] * avg_weights_grammar[i]
        pseudo_list_cv[fold_n][:, 5] += preds_list[i, :, 5] * avg_weights_conventions[i]
    print(pseudo_list_cv[fold_n][:10])
    
    ex = pd.read_csv("../data/input/past_fb_data.csv")
    cliped = np.clip(pseudo_list_cv[fold_n], 1, 5)
    ex["cohesion"] = cliped[:, 0]
    ex["syntax"] = cliped[:, 1]
    ex["vocabulary"] = cliped[:, 2]
    ex["phraseology"] = cliped[:, 3]
    ex["grammar"] = cliped[:, 4]
    ex["conventions"] = cliped[:, 5]
    ex.to_csv(f"../data/pseudo/pseudo_fold_{fold_n}.csv", index=False)

Global seed set to 42


[fold_3]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'mask_predictions.classifier.bias', 'mask_predi

reinit 1 layers


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at ../../weights/deberta_v3_base_512_reinit/last_epoch_fold3.ckpt
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

[[3.396 3.508 3.451 3.39  3.78  3.299]
 [3.23  3.35  3.447 3.346 3.426 3.176]
 [3.184 3.13  3.227 3.238 3.305 2.955]
 [3.586 3.668 3.688 3.549 3.703 3.703]
 [3.814 3.771 3.943 3.967 4.06  3.727]
 [2.729 2.668 3.09  2.994 3.074 2.662]
 [3.037 3.047 3.285 3.334 3.53  3.105]
 [4.12  4.17  4.215 4.203 4.473 4.113]
 [4.164 4.14  4.13  4.293 4.395 4.14 ]
 [4.094 4.08  4.227 4.21  4.367 4.016]]
[fold_4]


Some weights of the model checkpoint at microsoft/deberta-large were not used when initializing DebertaModel: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /notebooks/sr

reinit 1 layers


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at ../../weights/deberta_large_512_reinit/last_epoch_fold4.ckpt


Predicting: 0it [00:00, ?it/s]

In [19]:
for i in fold_list:
    ex = pd.read_csv("../data/input/past_fb_data.csv")
    cliped = np.clip(pseudo_list_cv[i], 1, 5)
    ex["cohesion"] = cliped[:, 0]
    ex["syntax"] = cliped[:, 1]
    ex["vocabulary"] = cliped[:, 2]
    ex["phraseology"] = cliped[:, 3]
    ex["grammar"] = cliped[:, 4]
    ex["conventions"] = cliped[:, 5]
    ex.to_csv(f"../data/pseudo/pseudo_fold_{i}.csv", index=False)

generate pseudo label (with optimize and blending)

In [13]:
# read external data
ex = pd.read_csv("../input/past-fb-data/past_fb_data.csv")

ex_train_X = ex["full_text"]

In [14]:
def one_fold_valid_pseudo(skf, cfg, valid_X, valid_y, ex_train_X, fold_n):
    print(f"[fold_{fold_n}]")
    seed_everything(cfg["general"]["seed"], workers=True)

    model = TransformersRegressorInference(cfg, f"{cfg['save_weight_folder']}/last_epoch_fold{fold_n}.ckpt")
    valid_preds = model.predict(valid_X)
    pseudo_preds = model.predict(ex_train_X)

    del model
    gc.collect()
    torch.cuda.empty_cache()

    score, scores = mcrmse(valid_y.values, valid_preds)
    print(f"mcrmse_score:{score}, mcrmse_scores:{scores}")

    return valid_preds, pseudo_preds

In [ ]:
fold_list = range(cfg1["general"]["n_splits"])
preds_list_cv = [[] for i in range(cfg1["general"]["n_splits"])]
pseudo_list_cv = [[] for i in range(cfg1["general"]["n_splits"])]
weights_list = [[] for _ in range(6)]
skf = MultilabelStratifiedKFold(
    n_splits=cfg1["general"]["n_splits"], shuffle=True, random_state=cfg1["general"]["seed"]
)
for j, fold_n in enumerate(fold_list):
    preds_list = []
    pseudo_list = []
    _, valid_indices = list(skf.split(train_X, train_y))[fold_n]
    valid_X_cv, valid_y_cv = (
        train_X.iloc[valid_indices].reset_index(drop=True),
        train_y.iloc[valid_indices].reset_index(drop=True),
    )

    for i, cfg in enumerate(cfg_list):
        valid_preds, pseudo_preds = one_fold_valid_pseudo(skf, cfg, valid_X_cv, valid_y_cv, ex_train_X, fold_n)
        preds_list.append(valid_preds)
        pseudo_list.append(pseudo_preds)
        del valid_preds
        gc.collect()

    score, scores = mcrmse(valid_y_cv.values, np.mean(preds_list, axis=0))
    print()
    print(f"simple_mean: mcrmse_score:{score}, mcrmse_scores:{scores}")
    for i, p in enumerate(preds_list):
        score, scores = mcrmse(valid_y_cv.values, p)
        print(f"cfg_{i}: mcrmse_score:{score}, mcrmse_scores:{scores}")

    preds_list = np.array(preds_list)
    pseudo_list = np.array(pseudo_list)
    preds_list_cv[j] = preds_list
    pseudo_list_cv[j] = pseudo_list
    cohesion = preds_list[:, :, 0]
    syntax = preds_list[:, :, 1]
    vocabulary = preds_list[:, :, 2]
    phraseology = preds_list[:, :, 3]
    grammar = preds_list[:, :, 4]
    conventions = preds_list[:, :, 5]
    target_list = [cohesion, syntax, vocabulary, phraseology, grammar, conventions]

    for t_idx, target in enumerate(target_list):

        def f(x):
            pred = np.zeros_like(target[0])
            for i, p in enumerate(target):
                pred += p * x[i]
            score = metrics.mean_squared_error(
                valid_y_cv.values[:, t_idx], pred, squared=False
            )
            return score

        init_state = np.ones((len(target))) / len(target)
        bounds = [(0.0, 1.0)] * len(target)
        result = minimize(f, init_state, method="Nelder-Mead", bounds=bounds)
        print(f"optimized_corr:{result['fun']}")

        weights = [[0] for _ in range(len(target))]
        for i in range(len(target)):
            weights[i] = result["x"][i]
        weights_list[t_idx].append(weights)
        print(f"weights:{weights}")

avg_weights_cohesion = np.mean(weights_list[0], axis=0)
avg_weights_syntax = np.mean(weights_list[1], axis=0)
avg_weights_vocabulary = np.mean(weights_list[2], axis=0)
avg_weights_phraseology = np.mean(weights_list[3], axis=0)
avg_weights_grammar = np.mean(weights_list[4], axis=0)
avg_weights_conventions = np.mean(weights_list[5], axis=0)

print()
print(f"cohesion averaged_weights:{avg_weights_cohesion}")
print(f"syntax averaged_weights:{avg_weights_syntax}")
print(f"vocabulary averaged_weights:{avg_weights_vocabulary}")
print(f"phraseology averaged_weights:{avg_weights_phraseology}")
print(f"grammar averaged_weights:{avg_weights_grammar}")
print(f"conventions averaged_weights:{avg_weights_conventions}")

[fold_0]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at ../input/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.bias

reinit 1 layers


Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

In [ ]:
score_cv = []
scores_cv = []
preds_list_valid_cv = [[] for i in range(cfg1["general"]["n_splits"])]
pseudo_list_valid_cv = [[] for i in range(cfg1["general"]["n_splits"])]
for j, (preds, pseudo) in enumerate(zip(preds_list_cv, pseudo_list_cv)):

    _, valid_indices = list(skf.split(train_X, train_y))[j]
    valid_X_cv, valid_y_cv = (
        train_X.iloc[valid_indices].reset_index(drop=True),
        train_y.iloc[valid_indices].reset_index(drop=True),
    )
    
    # blending
    preds_list_valid_cv[j] = np.zeros(preds[0].shape)
    pseudo_list_valid_cv[j] = np.zeros(pseudo[0].shape)
    for i in range(preds_list.shape[0]):
        preds_list_valid_cv[j][:, 0] += preds[i, :, 0] * avg_weights_cohesion[i]
        preds_list_valid_cv[j][:, 1] += preds[i, :, 1] * avg_weights_syntax[i]
        preds_list_valid_cv[j][:, 2] += preds[i, :, 2] * avg_weights_vocabulary[i]
        preds_list_valid_cv[j][:, 3] += preds[i, :, 3] * avg_weights_phraseology[i]
        preds_list_valid_cv[j][:, 4] += preds[i, :, 4] * avg_weights_grammar[i]
        preds_list_valid_cv[j][:, 5] += preds[i, :, 5] * avg_weights_conventions[i]
        
        pseudo_list_valid_cv[j][:, 0] += pseudo[i, :, 0] * avg_weights_cohesion[i]
        pseudo_list_valid_cv[j][:, 1] += pseudo[i, :, 1] * avg_weights_syntax[i]
        pseudo_list_valid_cv[j][:, 2] += pseudo[i, :, 2] * avg_weights_vocabulary[i]
        pseudo_list_valid_cv[j][:, 3] += pseudo[i, :, 3] * avg_weights_phraseology[i]
        pseudo_list_valid_cv[j][:, 4] += pseudo[i, :, 4] * avg_weights_grammar[i]
        pseudo_list_valid_cv[j][:, 5] += pseudo[i, :, 5] * avg_weights_conventions[i]
        
    score, scores = mcrmse(valid_y_cv.values, preds_list_valid_cv[j])
    print(f"optimized: mcrmse_score:{score}, mcrmse_scores:{scores}")
    score_cv.append(score)
    scores_cv.append(scores)

print(f"optimized mean_cv score:{np.mean(score_cv)}")
print(f"optimized mean_cv score:{np.mean(scores_cv, axis=0)}")

In [ ]:
for i in range(cfg1["general"]["n_splits"]):
    ex = pd.read_csv("../data/input/past_fb_data.csv")
    ex["cohesion"] = pseudo_list_valid_cv[i][:, 0]
    ex["syntax"] = pseudo_list_valid_cv[i][:, 1]
    ex["vocabulary"] = pseudo_list_valid_cv[i][:, 2]
    ex["phraseology"] = pseudo_list_valid_cv[i][:, 3]
    ex["grammar"] = pseudo_list_valid_cv[i][:, 4]
    ex["conventions"] = pseudo_list_valid_cv[i][:, 5]
    ex.to_csv(f"pseudo_fold_{i}.csv", index=False)